In [2]:
import sys, os

# Tell Python to also look one folder up (where 'db' lives)
sys.path.append(os.path.abspath(".."))

# Optional: confirm it worked
print(" Project path added.")

 Project path added.


In [5]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from pathlib import Path

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch existing foreign keys
cursor.execute("SELECT course_id FROM Courses;")
course_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT topic_type_id FROM topic_types;")
topic_type_ids = [row[0] for row in cursor.fetchall()]

print(f" Loaded {len(course_ids)} courses, {len(user_ids)} users, {len(topic_type_ids)} topic types")

# Generate 100 random topics
topics = []
for _ in range(100):
    course_id = random.choice(course_ids)
    user_id = random.choice(user_ids)
    topic_type_id = random.choice(topic_type_ids)
    topic_name = fake.sentence(nb_words=random.randint(2, 5)).replace(".", "")
    topics.append((course_id, user_id, topic_name, topic_type_id))

# Save CSV one level up
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "topics.csv"

df_topics = pd.DataFrame(topics, columns=["course_id", "user_id", "topic_name", "topic_type_id"])
df_topics.to_csv(csv_path, index=False, encoding="utf-8")

print(f" Saved 100 topics ")

# Insert generated topics into the database
df = pd.read_csv(csv_path) 

insert_query = """
INSERT INTO topics (course_id, user_id, topic_name, topic_type_id)
VALUES (%s, %s, %s, %s);
"""

count = 0
for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f" Inserted {count} rows into topics table.")


 Loaded 1699 courses, 100 users, 10 topic types
 Saved 100 topics 
 Inserted 100 rows into topics table.


In [6]:
cursor.execute("SELECT topic_id, topic_name, course_id, user_id, topic_type_id, created_at FROM topics LIMIT 10;")
for row in cursor.fetchall():
    print(row)


(1, 'Issue', 1650, 55, 6, datetime.datetime(2025, 11, 15, 2, 16, 4))
(2, 'Us thought', 1275, 10, 8, datetime.datetime(2025, 11, 15, 2, 16, 4))
(3, 'Yet rock compare', 59, 39, 8, datetime.datetime(2025, 11, 15, 2, 16, 4))
(4, 'Reveal phone never', 1446, 96, 8, datetime.datetime(2025, 11, 15, 2, 16, 4))
(5, 'Talk so', 163, 48, 10, datetime.datetime(2025, 11, 15, 2, 16, 4))
(6, 'Growth political after wait door', 640, 100, 7, datetime.datetime(2025, 11, 15, 2, 16, 4))
(7, 'Find base actually', 1682, 88, 1, datetime.datetime(2025, 11, 15, 2, 16, 4))
(8, 'Old view home', 876, 97, 8, datetime.datetime(2025, 11, 15, 2, 16, 4))
(9, 'Wonder just forget', 467, 86, 9, datetime.datetime(2025, 11, 15, 2, 16, 4))
(10, 'Try perform', 1369, 45, 5, datetime.datetime(2025, 11, 15, 2, 16, 4))


In [ ]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch existing foreign keys
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT course_id FROM Courses;")
course_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT topic_id FROM topics;")
topic_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT category_id FROM task_categories;")
category_ids = [row[0] for row in cursor.fetchall()]

print(f"📊 Loaded {len(user_ids)} users, {len(course_ids)} courses, {len(topic_ids)} topics, {len(category_ids)} categories")

# Generate 100 random tasks
statuses = ['todo', 'in_progress', 'done']
tasks = []

for _ in range(100):
    user_id = random.choice(user_ids)
    course_id = random.choice(course_ids)
    topic_id = random.choice(topic_ids)
    category_id = random.choice(category_ids)

    title = fake.sentence(nb_words=random.randint(3, 6)).replace(".", "")
    due_date = datetime.now() + timedelta(days=random.randint(1, 30))
    status = random.choice(statuses)
    priority = random.randint(1, 5)

    tasks.append((user_id, course_id, topic_id, title, due_date, status, priority, category_id))

# Save to ../data/Raw_data/
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "tasks.csv"

df_tasks = pd.DataFrame(tasks, columns=[
    "user_id", "course_id", "topic_id", "title", "due_date", "status", "priority", "category_id"
])
df_tasks.to_csv(csv_path, index=False, encoding="utf-8")

print(f" Saved 100 tasks")

# Insert into the database
insert_query = """
INSERT INTO tasks (user_id, course_id, topic_id, title, due_date, status, priority, category_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

count = 0
for _, row in df_tasks.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f" Inserted {count} rows into tasks table.")


In [8]:
cursor.execute("SELECT task_id, title, status, due_date, priority FROM tasks LIMIT 10;")
for row in cursor.fetchall():
    print(row)


(1, 'Power policy wrong teacher very agency democratic performance', 'todo', datetime.datetime(2025, 11, 19, 2, 20, 10), 5)
(2, 'That with care', 'done', datetime.datetime(2025, 12, 3, 2, 20, 10), 1)
(3, 'Production per new watch', 'todo', datetime.datetime(2025, 11, 27, 2, 20, 10), 1)
(4, 'Road total', 'done', datetime.datetime(2025, 12, 14, 2, 20, 10), 2)
(5, 'Upon know to military tell when', 'done', datetime.datetime(2025, 12, 14, 2, 20, 10), 5)
(6, 'Choice region glass', 'todo', datetime.datetime(2025, 11, 28, 2, 20, 10), 5)
(7, 'Once art interview moment senior collection', 'done', datetime.datetime(2025, 11, 30, 2, 20, 10), 3)
(8, 'Charge onto fear', 'todo', datetime.datetime(2025, 11, 22, 2, 20, 10), 1)
(9, 'Entire boy', 'todo', datetime.datetime(2025, 11, 20, 2, 20, 10), 5)
(10, 'Window effort their say whole', 'todo', datetime.datetime(2025, 11, 20, 2, 20, 10), 2)


In [3]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# --- Fetch foreign keys
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT topic_id FROM topics;")
topic_ids = [row[0] for row in cursor.fetchall()]

print(f" Loaded {len(user_ids)} users and {len(topic_ids)} topics.")

# --- Generate 100 focus sessions
techniques = ["pomodoro", "flowtime", "custom"]
session_types = ["solo", "group"]
sessions = []

for _ in range(100):
    host_id = random.choice(user_ids)
    topic_id = random.choice(topic_ids)
    technique_type = random.choice(techniques)
    session_type = random.choice(session_types)

    # Random start time within the past 30 days
    start_time = datetime.now() - timedelta(days=random.randint(0, 30),
                                            hours=random.randint(0, 23),
                                            minutes=random.randint(0, 59))
    duration = random.randint(15, 120)   # minutes
    end_time = start_time + timedelta(minutes=duration)

    # Breaks (short 5-10 min, long 15-25 min)
    short_break = random.choice([5, 10, 15])
    long_break = random.choice([15, 20, 25])

    sessions.append((
        host_id,
        start_time,
        end_time,
        topic_id,
        technique_type,
        short_break,
        long_break,
        session_type
    ))

# --- Save CSV one level up → ../data/Raw_data/
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "timersessions.csv"

df_sessions = pd.DataFrame(
    sessions,
    columns=[
        "host_id",
        "start_time",
        "end_time",
        "topic_id",
        "technique_type",
        "short_break_min",
        "long_break_min",
        "session_type"
    ]
)
df_sessions.to_csv(csv_path, index=False, encoding="utf-8")

print(f" Saved 100 timer sessions")

# --- Insert into the database
insert_query = """
INSERT INTO timersessions (
    host_id, start_time, end_time, topic_id,
    technique_type, short_break_min, long_break_min, session_type
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

count = 0
for _, row in df_sessions.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f" Inserted {count} rows into timersessions table.")


MySQL connector is working!
 Loaded 100 users and 100 topics.
 Saved 100 timer sessions
 Inserted 100 rows into timersessions table.


In [4]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from pathlib import Path

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch existing foreign keys
cursor.execute("SELECT timer_id FROM timersessions;")
timer_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

print(f"Loaded {len(timer_ids)} timer sessions and {len(user_ids)} users.")

# Generate 100 random participant entries
records = set()  # prevent duplicates
participants = []

while len(participants) < 100:
    timer_id = random.choice(timer_ids)
    user_id = random.choice(user_ids)
    if (timer_id, user_id) in records:
        continue  # avoid duplicate primary key
    records.add((timer_id, user_id))
    role = random.choice(['host', 'member'])
    joined_at = fake.date_time_this_year()
    participants.append((timer_id, user_id, joined_at, role))

# Save to CSV one level up
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "sessionparticipants.csv"

df_participants = pd.DataFrame(participants, columns=["timer_id", "user_id", "joined_at", "role"])
df_participants.to_csv(csv_path, index=False, encoding="utf-8")

print(f"Saved {len(df_participants)} session participants.")

# Insert into DB
insert_query = """
INSERT INTO sessionparticipants (timer_id, user_id, joined_at, role)
VALUES (%s, %s, %s, %s);
"""

count = 0
for _, row in df_participants.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f"Inserted {count} rows into sessionparticipants table.")


Loaded 100 timer sessions and 100 users.
Saved 100 session participants.
Inserted 100 rows into sessionparticipants table.


In [5]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from pathlib import Path
from datetime import timedelta

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch task IDs
cursor.execute("SELECT task_id FROM tasks;")
task_ids = [row[0] for row in cursor.fetchall()]

print(f"Loaded {len(task_ids)} tasks.")

# Generate 100 reminders
reminders = []
for _ in range(100):
    task_id = random.choice(task_ids)
    # reminder_time: within next 30 days
    reminder_time = fake.date_time_between(start_date="now", end_date="+30d")
    method = random.choice(['in_app', 'email', 'sms'])
    reminders.append((task_id, reminder_time, method))

# Save CSV one level up
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "reminders.csv"

df_reminders = pd.DataFrame(reminders, columns=["task_id", "reminder_time", "method"])
df_reminders.to_csv(csv_path, index=False, encoding="utf-8")

print(f"Saved {len(df_reminders)} reminders.")

# Insert into DB
insert_query = """
INSERT INTO reminders (task_id, reminder_time, method)
VALUES (%s, %s, %s);
"""

count = 0
for _, row in df_reminders.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f"Inserted {count} rows into reminders table.")


Loaded 100 tasks.
Saved 100 reminders.
Inserted 100 rows into reminders table.


In [6]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from pathlib import Path

# Initialize
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch existing item types
cursor.execute("SELECT item_type_id, type_name FROM item_types;")
item_types = cursor.fetchall()

print(f"Loaded {len(item_types)} item types.")

# Helper data
rarities = ['common', 'rare', 'epic', 'legendary']

# Generate 100 focus items
focus_items = []
for _ in range(100):
    item_type_id, type_name = random.choice(item_types)
    item_name = f"{fake.word().capitalize()} {type_name}"
    image_url = f"https://example.com/images/{item_name.replace(' ', '_').lower()}.png"
    rarity = random.choices(rarities, weights=[60, 25, 10, 5])[0]  # weighted probabilities
    focus_cost_min = random.randint(50, 500)
    description = fake.sentence(nb_words=6).replace(".", "")
    focus_items.append((item_type_id, item_name, image_url, rarity, focus_cost_min, description))

# Save to CSV 
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "focusitems.csv"

df_items = pd.DataFrame(focus_items, columns=[
    "item_type_id", "item_name", "image_url", "rarity_level", "focus_cost_min", "description"
])
df_items.to_csv(csv_path, index=False, encoding="utf-8")

print(f"Saved {len(df_items)} focus items")

# --- Insert into DB
insert_query = """
INSERT INTO focusitems (item_type_id, item_name, image_url, rarity_level, focus_cost_min, description)
VALUES (%s, %s, %s, %s, %s, %s);
"""

count = 0
for _, row in df_items.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f"Inserted {count} rows into focusitems table.")


Loaded 25 item types.
Saved 100 focus items
Inserted 100 rows into focusitems table.


In [18]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path
import json


# Initialize Faker and DB connection
fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch existing foreign key data (Users + Focus Items)
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT item_id FROM focusitems;")
item_ids = [row[0] for row in cursor.fetchall()]

print(f" Loaded {len(user_ids)} users and {len(item_ids)} focus items.")

# Generate Daily Focus Logs
logs = []
for _ in range(100):
    user_id = random.choice(user_ids)
    focus_date = fake.date_between(start_date="-30d", end_date="today")

    total_sessions = random.randint(1, 8)
    total_focus_min = total_sessions * random.randint(20, 60)

    # Generate JSON fields (start times + topics)
    focus_start_times = [fake.time() for _ in range(total_sessions)]
    focus_topics = [fake.word() for _ in range(random.randint(2, 4))]

    # Convert to JSON-safe strings
    focus_start_times_json = json.dumps(focus_start_times, ensure_ascii=False)
    focus_topics_json = json.dumps(focus_topics, ensure_ascii=False)

    # Reward system (like Forest app):
    # Users earn better items for longer total focus minutes
    if total_focus_min < 60:
        item_earned_id = random.choice(item_ids[:25]) if len(item_ids) >= 25 else random.choice(item_ids)
    elif total_focus_min < 180:
        item_earned_id = random.choice(item_ids[25:50]) if len(item_ids) >= 50 else random.choice(item_ids)
    elif total_focus_min < 360:
        item_earned_id = random.choice(item_ids[50:75]) if len(item_ids) >= 75 else random.choice(item_ids)
    else:
        item_earned_id = random.choice(item_ids[75:]) if len(item_ids) > 75 else random.choice(item_ids)

    logs.append((
        user_id,
        focus_date,
        total_sessions,
        total_focus_min,
        focus_start_times_json,
        focus_topics_json,
        item_earned_id
    ))

# Save as CSV (for backup and inspection)
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "dailyfocuslog.csv"

df_logs = pd.DataFrame(logs, columns=[
    "user_id", "focus_date", "total_sessions", "total_focus_min",
    "focus_start_times", "focus_topics", "item_earned_id"
])

# Validate data before inserting
nan_summary = df_logs.isna().sum()
print("\n NaN counts per column:\n", nan_summary)

assert not df_logs.isna().any().any(), " DataFrame contains NaN values!"
assert df_logs["focus_start_times"].str.len().min() > 0, " Empty JSON strings found!"

df_logs.to_csv(csv_path, index=False, encoding="utf-8")
print(f"\n Saved {len(df_logs)} clean daily focus logs.")

# Insert into MySQL database
insert_query = """
INSERT INTO dailyfocuslog 
(user_id, focus_date, total_sessions, total_focus_min, focus_start_times, focus_topics, item_earned_id)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

count = 0
for _, row in df_logs.iterrows():
    cursor.execute(insert_query, tuple(row))
    count += 1

connection.commit()
print(f" Inserted {count} rows into dailyfocuslog table successfully.")

# Clean up
cursor.close()
connection.close()
print(" Database connection closed.\n")
print(" Daily focus log generation complete.")


 Loaded 100 users and 100 focus items.

 NaN counts per column:
 user_id              0
focus_date           0
total_sessions       0
total_focus_min      0
focus_start_times    0
focus_topics         0
item_earned_id       0
dtype: int64

 Saved 100 clean daily focus logs.
 Inserted 100 rows into dailyfocuslog table successfully.
 Database connection closed.

 Daily focus log generation complete.


In [23]:
from db.connect_db import get_connection
import pandas as pd
from collections import Counter
from datetime import datetime
import random


# Connect to database
connection = get_connection()
cursor = connection.cursor()

# Load daily focus logs
query = """
SELECT user_id, focus_date, total_sessions, total_focus_min, item_earned_id, focus_topics
FROM dailyfocuslog;
"""
df = pd.read_sql(query, connection)
print(f"Loaded {len(df)} daily focus records from database.")

# Load item and type info
cursor.execute("""
    SELECT fi.item_id, fi.item_name, it.type_name
    FROM focusitems fi
    JOIN item_types it ON fi.item_type_id = it.item_type_id;
""")
item_info = {row[0]: {"name": row[1], "type": row[2]} for row in cursor.fetchall()}
print(f"Loaded {len(item_info)} focus items with their types.")

# Load topic mapping (assuming topics table exists)
cursor.execute("SELECT topic_id, topic_name FROM topics;")
topic_map = {name: tid for tid, name in cursor.fetchall()}

# Compute stats per user
stats = []
for user_id, group in df.groupby("user_id"):
    total_sessions = int(group["total_sessions"].sum())
    total_focus_time_min = int(group["total_focus_min"].sum())
    avg_duration_min = round(total_focus_time_min / total_sessions, 2) if total_sessions else 0.0

    # Streaks
    dates = sorted(pd.to_datetime(group["focus_date"]).unique())
    longest_streak = current_streak = 1
    for i in range(1, len(dates)):
        if (dates[i] - dates[i - 1]).days == 1:
            current_streak += 1
            longest_streak = max(longest_streak, current_streak)
        else:
            current_streak = 1

    last_session_at = pd.to_datetime(group["focus_date"]).max()

    # Favorite item
    most_common_item_id = (
        int(group["item_earned_id"].mode().iloc[0])
        if not group["item_earned_id"].isna().all()
        else None
    )

    favorite_item_type = (
        item_info[most_common_item_id]["type"]
        if most_common_item_id in item_info
        else None
    )
    favorite_item_name = (
        item_info[most_common_item_id]["name"]
        if most_common_item_id in item_info
        else None
    )

    # Most frequent topic
    all_topics = []
    for tlist in group["focus_topics"]:
        try:
            topics = eval(tlist) if isinstance(tlist, str) else []
            all_topics.extend(topics)
        except Exception:
            continue

    most_common_topic_name = (
        Counter(all_topics).most_common(1)[0][0] if all_topics else None
    )
    most_frequent_topic = (
        int(topic_map.get(most_common_topic_name))
        if most_common_topic_name in topic_map
        else None
    )

    stats.append((
        int(user_id),
        total_sessions,
        total_focus_time_min,
        float(avg_duration_min),
        int(longest_streak),
        int(current_streak),
        last_session_at.to_pydatetime() if pd.notna(last_session_at) else None,
        str(favorite_item_type) if favorite_item_type else None,
        str(favorite_item_name) if favorite_item_name else None,
        most_frequent_topic
    ))

# Insert results into `studystats`

insert_query = """
INSERT INTO studystats (
    user_id, total_sessions, total_focus_time_min, avg_duration_min,
    longest_streak_days, current_streak_days, last_session_at,
    favorite_item_type, favorite_item_name, most_frequent_topic
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
    total_sessions=VALUES(total_sessions),
    total_focus_time_min=VALUES(total_focus_time_min),
    avg_duration_min=VALUES(avg_duration_min),
    longest_streak_days=VALUES(longest_streak_days),
    current_streak_days=VALUES(current_streak_days),
    last_session_at=VALUES(last_session_at),
    favorite_item_type=VALUES(favorite_item_type),
    favorite_item_name=VALUES(favorite_item_name),
    most_frequent_topic=VALUES(most_frequent_topic);
"""

cursor.executemany(insert_query, stats)
connection.commit()

print(f" Inserted or updated {len(stats)} user study stats successfully.")

cursor.close()
connection.close()


Loaded 100 daily focus records from database.
Loaded 100 focus items with their types.
 Inserted or updated 68 user study stats successfully.


/var/folders/xv/rz8bm0912bn7rcqqwsgsxtsc0000gn/T/ipykernel_80133/1231412668.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [25]:
from db.connect_db import get_connection
import pandas as pd
from datetime import datetime


connection = get_connection()
cursor = connection.cursor()

# Fetch earned items from dailyfocuslog
query = """
SELECT user_id, item_earned_id, MAX(focus_date) AS last_date, COUNT(*) AS earned_count
FROM dailyfocuslog
WHERE item_earned_id IS NOT NULL
GROUP BY user_id, item_earned_id;
"""
df = pd.read_sql(query, connection)
print(f"Loaded {len(df)} earned item records from dailyfocuslog.")

# Prepare data for insertion
records = []
for _, row in df.iterrows():
    user_id = int(row["user_id"])
    item_id = int(row["item_earned_id"])
    quantity = int(row["earned_count"])
    last_earned_at = pd.to_datetime(row["last_date"]).to_pydatetime()

    records.append((user_id, item_id, quantity, last_earned_at, False))

# Insert or update into userfocusitems
insert_query = """
INSERT INTO userfocusitems (user_id, item_id, quantity, last_earned_at, is_placed)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    quantity = quantity + VALUES(quantity),
    last_earned_at = GREATEST(last_earned_at, VALUES(last_earned_at));
"""

cursor.executemany(insert_query, records)
connection.commit()

print(f" Inserted or updated {len(records)} user inventory records successfully.")

cursor.close()
connection.close()


Loaded 99 earned item records from dailyfocuslog.
 Inserted or updated 99 user inventory records successfully.


/var/folders/xv/rz8bm0912bn7rcqqwsgsxtsc0000gn/T/ipykernel_80133/480602331.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [26]:
from db.connect_db import get_connection
import pandas as pd
import random
from pathlib import Path
from datetime import datetime


connection = get_connection()
cursor = connection.cursor()

# Load user-owned items (from userfocusitems)
query = """
SELECT user_id, item_id, quantity
FROM userfocusitems;
"""
df = pd.read_sql(query, connection)

print(f"Loaded {len(df)} user–item ownership records.")

# Generate layout placements
placements = []

for _, row in df.iterrows():
    user_id = int(row["user_id"])
    item_id = int(row["item_id"])
    quantity = int(row["quantity"])

    # Decide how many items to place (at least 1, at most quantity)
    num_places = max(1, min(quantity, random.randint(1, quantity)))

    for _ in range(num_places):
        position_x = random.randint(0, 50)  # example grid width
        position_y = random.randint(0, 50)  # example grid height

        placements.append((
            user_id,
            item_id,
            position_x,
            position_y,
            datetime.now()
        ))

print(f"Generated {len(placements)} total placements for user city layouts.")

# Save CSV to ../data/Raw_data/
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / "usercitylayout.csv"

df_place = pd.DataFrame(placements, columns=[
    "user_id", "item_id", "position_x", "position_y", "placed_at"
])
df_place.to_csv(csv_path, index=False, encoding="utf-8")

print(f" Saved {len(df_place)} layout records to usercitylayout.csv.")

# Insert into database
insert_query = """
INSERT INTO usercitylayout (user_id, item_id, position_x, position_y, placed_at)
VALUES (%s, %s, %s, %s, %s);
"""

cursor.executemany(insert_query, placements)
connection.commit()

print(f" Inserted {len(placements)} rows into usercitylayout successfully.")

cursor.close()
connection.close()


Loaded 99 user–item ownership records.
Generated 100 total placements for user city layouts.
 Saved 100 layout records to usercitylayout.csv.
 Inserted 100 rows into usercitylayout successfully.


/var/folders/xv/rz8bm0912bn7rcqqwsgsxtsc0000gn/T/ipykernel_80133/2233827405.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [29]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import date, timedelta
from pathlib import Path
import unicodedata
import re

fake = Faker()
connection = get_connection()
cursor = connection.cursor()

# Fetch reward items
cursor.execute("SELECT item_id FROM focusitems;")
item_ids = [row[0] for row in cursor.fetchall()]

print(f"Loaded {len(item_ids)} reward items.")

# Allowed chars only
allowed_regex = re.compile(r"^[A-Za-z0-9 ,.\-?!()]{3,150}$")
strip_illegal = re.compile(r"[^A-Za-z0-9 ,.\-?!()]")

def clean_ascii(text):
    """Remove unicode, enforce allowed characters."""
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = strip_illegal.sub("", text)
    text = re.sub(r"\s+", " ", text).strip()
    if len(text) < 3:
        text = "Focus Challenge"
    return text[:140]

keywords = [
    "Focus Blitz", "Study Marathon", "Brain Boost", "Zen Challenge",
    "Deep Work Sprint", "No Distractions", "Knowledge Quest",
    "Learning Rush", "Mind Growth", "Pomodoro Push",
    "Exam Prep Surge", "Discipline Drive", "Mastery Month"
]

def random_month_range():
    start = fake.date_between(start_date="-6M", end_date="+1M")
    end = start + timedelta(days=random.randint(28, 35))
    return start, end

rows = []
bad_rows = []

for _ in range(100):
    raw_title = f"{random.choice(keywords)} {random.randint(1, 300)}"
    title = clean_ascii(raw_title)

    # Force validity
    if not allowed_regex.match(title):
        title = f"Focus Challenge {random.randint(1,999)}"

    description = clean_ascii(fake.sentence(nb_words=10).replace(".", ""))

    start_date, end_date = random_month_range()
    reward_item_id = random.choice(item_ids)

    # Validate BEFORE writing to DB
    if not allowed_regex.match(title):
        bad_rows.append(("TITLE", title))
        continue
    if len(description) > 255:
        bad_rows.append(("DESC_TOO_LONG", description))
        continue

    rows.append((title, description, start_date, end_date, 600, reward_item_id))

print(f"\nValid rows: {len(rows)} | Bad rows skipped: {len(bad_rows)}")
if bad_rows:
    print("Examples of bad rows:", bad_rows[:5])

# Save CSV
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)

csv_path = output_dir / "monthlychallenges.csv"
pd.DataFrame(rows, columns=[
    "title", "description", "start_date", "end_date", "goal_minutes", "reward_item_id"
]).to_csv(csv_path, index=False)

print("\nCSV saved.")


# INSERT ROW BY ROW SAFE MODE
insert_query = """
INSERT INTO monthlychallenges
(title, description, start_date, end_date, goal_minutes, reward_item_id)
VALUES (%s, %s, %s, %s, %s, %s);
"""

inserted = 0
failed = 0

for r in rows:
    try:
        cursor.execute(insert_query, r)
        inserted += 1
    except Exception as e:
        print("\n FAILED ROW:", r)
        print("MYSQL ERROR:", e)
        failed += 1

connection.commit()
cursor.close()
connection.close()

print(f"\nInserted {inserted} rows successfully.")
print(f"Failed rows: {failed}")


Loaded 100 reward items.

Valid rows: 100 | Bad rows skipped: 0

CSV saved.

Inserted 100 rows successfully.
Failed rows: 0


In [30]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path

fake = Faker()

# Connect to DB
connection = get_connection()
cursor = connection.cursor()

# Fetch foreign keys
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT challenge_id, goal_minutes FROM monthlychallenges;")
challenges = cursor.fetchall()  # list of (challenge_id, goal_minutes)

print(f"Loaded {len(user_ids)} users and {len(challenges)} challenges.")

rows = []
used_pairs = set()  # to enforce PRIMARY KEY(user_id, challenge_id)

# Generate exactly 100 unique user-challenge progress logs
while len(rows) < 100:

    user_id = random.choice(user_ids)
    challenge_id, goal_minutes = random.choice(challenges)

    pair = (user_id, challenge_id)
    if pair in used_pairs:
        continue  # avoid violating composite PK

    used_pairs.add(pair)

    # Progress
    total_minutes = random.randint(0, goal_minutes + 200)

    is_completed = total_minutes >= goal_minutes

    # Completion date only if completed
    if is_completed:
        completed_at = fake.date_time_between(start_date="-30d", end_date="now")
    else:
        completed_at = None

    rows.append((
        user_id,
        challenge_id,
        total_minutes,
        is_completed,
        completed_at
    ))

print(f"\nGenerated {len(rows)} unique user-challenge-progress rows.")


# Save CSV → ../data/Raw_data/userchallengeprogress.csv
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)

csv_path = output_dir / "userchallengeprogress.csv"

df = pd.DataFrame(rows, columns=[
    "user_id", "challenge_id", "total_minutes",
    "is_completed", "completed_at"
])

df.to_csv(csv_path, index=False, encoding="utf-8")
print(f"✔ Saved CSV → {csv_path}")


# Insert into MySQL row-by-row (safe)
insert_query = """
INSERT INTO userchallengeprogress
(user_id, challenge_id, total_minutes, is_completed, completed_at)
VALUES (%s, %s, %s, %s, %s);
"""

inserted = 0
failed = 0

for r in rows:
    try:
        cursor.execute(insert_query, r)
        inserted += 1
    except Exception as e:
        print("\n Failed row:", r)
        print("Error:", e)
        failed += 1

connection.commit()
cursor.close()
connection.close()

print(f"\n✔ Inserted {inserted} rows successfully.")
print(f" Failed inserts: {failed}")


Loaded 100 users and 100 challenges.

Generated 100 unique user-challenge-progress rows.
✔ Saved CSV → ../data/Raw_data/userchallengeprogress.csv

✔ Inserted 100 rows successfully.
 Failed inserts: 0


In [31]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path

fake = Faker()

# Connect to DB
connection = get_connection()
cursor = connection.cursor()


# FETCH FOREIGN KEYS
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT timer_id FROM timersessions;")
timer_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT mood_level_id FROM mood_levels;")
mood_levels = [row[0] for row in cursor.fetchall()]

print(f"Loaded: {len(user_ids)} users, {len(timer_ids)} timer sessions, {len(mood_levels)} mood levels.")

# GENERATE 100 MOOD ENTRIES
rows = []

for _ in range(100):

    user_id = random.choice(user_ids)

    # 70% chance mood is linked to a timer session, 30% chance standalone
    timer_id = random.choice(timer_ids) if random.random() < 0.7 else None

    mood_level_id = random.choice(mood_levels)

    # Sometimes user writes a short note
    note = fake.sentence(nb_words=random.randint(4, 12))
    note = note[:250]  # ensure <255 chars for safety

    # Recorded at: random time within last 30 days
    recorded_at = fake.date_time_between(start_date="-30d", end_date="now")

    rows.append((
        user_id,
        timer_id,
        mood_level_id,
        note,
        recorded_at
    ))

print(f"\nGenerated {len(rows)} mood entries.")

# SAVE CSV
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)

csv_path = output_dir / "moodtracking.csv"

df = pd.DataFrame(rows, columns=[
    "user_id", "timer_id", "mood_level_id", "note", "recorded_at"
])

df.to_csv(csv_path, index=False, encoding="utf-8")
print(f"✔ Saved moodtracking.csv with {len(df)} rows.")


# INSERT INTO MySQL — row-by-row for safety
insert_query = """
INSERT INTO moodtracking
(user_id, timer_id, mood_level_id, note, recorded_at)
VALUES (%s, %s, %s, %s, %s);
"""

inserted = 0
failed = 0

for r in rows:
    try:
        cursor.execute(insert_query, r)
        inserted += 1
    except Exception as e:
        print("\n Failed row:", r)
        print("Error:", e)
        failed += 1

connection.commit()
cursor.close()
connection.close()

print(f"\n✔ Successfully inserted {inserted} rows into moodtracking.")
print(f" Failed inserts: {failed}")


Loaded: 100 users, 100 timer sessions, 5 mood levels.

Generated 100 mood entries.
✔ Saved moodtracking.csv with 100 rows.

✔ Successfully inserted 100 rows into moodtracking.
 Failed inserts: 0


In [32]:
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from datetime import datetime
from pathlib import Path

fake = Faker()

# Connect
connection = get_connection()
cursor = connection.cursor()

# FETCH USERS
cursor.execute("SELECT user_id FROM Users;")
user_ids = [row[0] for row in cursor.fetchall()]

print(f"Loaded {len(user_ids)} users.")

# Leaderboard periods
periods = ["daily", "weekly", "monthly", "all_time"]

rows = []


# GENERATE LEADERBOARD STATS
for user_id in user_ids:

    # Base performance for all_time
    base_total_focus = random.randint(500, 8000)
    base_total_sessions = base_total_focus // random.randint(20, 60)
    base_streak = random.randint(1, 40)

    # Create entries for each period
    for period in periods:

        if period == "daily":
            total_focus = random.randint(0, 240)                      # up to 4 hours
            total_sessions = max(1, total_focus // random.randint(20, 45))
            streak = random.randint(0, 5)

        elif period == "weekly":
            total_focus = random.randint(200, 1200)
            total_sessions = max(1, total_focus // random.randint(20, 45))
            streak = random.randint(1, 15)

        elif period == "monthly":
            total_focus = random.randint(800, 5000)
            total_sessions = max(1, total_focus // random.randint(20, 45))
            streak = random.randint(5, 30)

        else:  # all_time
            total_focus = base_total_focus
            total_sessions = base_total_sessions
            streak = base_streak

        rows.append((
            user_id,
            total_focus,
            total_sessions,
            streak,
            period,
            datetime.now()
        ))

print(f"\nGenerated {len(rows)} leaderboard entries.")


# SAVE CSV
output_dir = Path("..") / "data" / "Raw_data"
output_dir.mkdir(parents=True, exist_ok=True)

csv_path = output_dir / "leaderboardstats.csv"

df = pd.DataFrame(rows, columns=[
    "user_id", "total_focus_min", "total_sessions",
    "streak_days", "period_type", "updated_at"
])

df.to_csv(csv_path, index=False, encoding="utf-8")
print(f"✔ Saved leaderboardstats.csv with {len(df)} rows.")


# INSERT INTO DATABASE
insert_query = """
INSERT INTO leaderboardstats
(user_id, total_focus_min, total_sessions, streak_days, period_type, updated_at)
VALUES (%s, %s, %s, %s, %s, %s);
"""

inserted = 0
failed = 0

for r in rows:
    try:
        cursor.execute(insert_query, r)
        inserted += 1
    except Exception as e:
        print("\n Failed row:", r)
        print("Error:", e)
        failed += 1

connection.commit()
cursor.close()
connection.close()

print(f"\n✔ Inserted {inserted} leaderboard rows successfully.")
print(f" Failed inserts: {failed}")


Loaded 100 users.

Generated 400 leaderboard entries.
✔ Saved leaderboardstats.csv with 400 rows.

✔ Inserted 400 leaderboard rows successfully.
 Failed inserts: 0
